In [4]:
import numpy as np
import pickle
from pathlib import Path
from os import walk
import nrrd

## Helper code for reading all nrrds from folder

In [10]:
def get_seqs(path = "input_dir"):
    segmentation_files = []
    sequence_files = []
    for (root, direc, files) in walk(path):
        for file in files:
            if not "invivo" in root or not "invivo" in direc or not "invivo" in file:
                if file.endswith("label.nrrd") or file.endswith("seg.nrrd"):
                    segmentation_files.append(Path(root)/Path(file))
                if not file.endswith("label.nrrd") and not file.endswith("seg.nrrd") and file.endswith(".nrrd"):
                    sequence_files.append(Path(root)/Path(file))
    segmentation_files.sort(), sequence_files.sort()
    # for seq, seg in zip(sequence_files, segmentation_files):
    #     print(seq, seg)
    sequence_files = [nrrd.read(file)[0] for file in sequence_files]
    segmentation_files = [nrrd.read(file)[0] for file in segmentation_files]
    # for seq in sequence_files:
    #     print(seq.shape)
    return sequence_files, segmentation_files

def transpose(seqs, segs):
    new_seq = []
    new_seg = []
    for seq in seqs:
        if seq.shape[0] > seq.shape[1]:
            new_seq.append(seq.transpose((1, 0, 2)))
        elif seq.shape[0] < seq.shape[1]:
            new_seq.append(seq)
        elif seq.shape[0] == seq.shape[1]:
            new_seq.append(seq)
        else:
            print("Something wrong")
    for seg in segs:
        if seg.shape[0] > seg.shape[1]:
            new_seq.append(seg.transpose((1, 0, 2)))
        elif seg.shape[0] < seg.shape[1]:
            new_seg.append(seg)
        elif seg.shape[0] == seg.shape[1]:
            new_seg.append(seg)
        else:
            print("Something wrong")
    return new_seq, new_seg


def crop_to_size(seqs, segs, x, y):
    return seqs[:,0:x,0:y,:], segs[:,0:x,0:y,:]

def pad_with_zeros(seqs, segs, pad_dim=(128, 128, 31)):
    new_seq = []
    new_seg = []

    for seq in seqs:
        pad = np.zeros(pad_dim)
        pad[0:seq.shape[0], 0:seq.shape[1], 0:seq.shape[2]] = seq
        new_seq.append(pad)
    
    for seg in segs:
        pad = np.zeros(pad_dim)
        pad[0:seg.shape[0], 0:seg.shape[1], 0:seg.shape[2]] = seg
        new_seg.append(pad)

    return new_seq, new_seg

def allow_only_by(seqs, segs, size=(128, 256, 20)):
    seqs = [seq for seq in seqs if seq.shape == size]
    segs = [seg for seg in segs if seg.shape == size]
    return seqs, segs


def convert_to_tensor(seqs, segs):
    # for seq, seg in zip(seqs, segs):
    #     print(seq.shape, seg.shape)
    seqs = np.concatenate(seqs, axis=2)
    segs = np.concatenate(segs, axis=2)
    seqs = seqs.transpose((2, 0, 1))
    segs = segs.transpose((2, 0, 1))
    seqs, segs = np.expand_dims(seqs, axis=-1), np.expand_dims(segs, axis=-1)
    return seqs, segs

def normalize_by(seq, min_v=0, max_v=1, std=0, stds=1, mean=0, clip=False, clip_max=False):
    if clip_max:
        seq = seq.clip(seq, max=clip_max)
    if clip:
        seq = seq.clip(seq, max = mean + (std * stds))
    seq = seq / max_v
    return seq

def train_test_validation(volume):
    pct_80 = int(volume.shape[0] / 100 * 80)
    pct_90 = int((volume.shape[0] / 100 * 90))
    pct_100 = int(volume.shape[0])
    return volume[0:pct_80], volume[pct_80: pct_90], volume[pct_90:pct_100]

def write_pickle(obj, file):
    with open(file, 'b+w') as pout:
        pickle.dump(obj, pout)

# ASL

In [3]:
input_dir = Path("../ROI/ROI_NEW/ASL")
output_file = Path("../data/pickles/ASL.pkl")
seqs, segs = get_seqs(input_dir)
seqs, segs = pad_with_zeros(seqs, segs, pad_dim=(128, 128, 31))
seqs, segs = convert_to_tensor(seqs, segs)
seqs = normalize_by(seqs, max_v=19076.6, std=3087.4, stds=6, mean=552.27, clip=True)
seg_train, seg_test, seg_val = train_test_validation(segs)
seq_train, seq_test, seq_val = train_test_validation(seqs)

ASL_data = {"seg_test": seg_test,
            "seq_test": seq_test,
            "seg_train": seg_train,
            "seq_train": seq_train,
            "seg_val": seg_val,
            "seq_val": seq_val}
write_pickle(ASL_data, output_file)

# DWI

In [52]:
output_file = Path("../data/pickles/DWI_data.pkl")
with open(output_file, "+br") as p_in:
    data = pickle.load(p_in)

# DWI_data = {"seg_test": seg_test,
#             "seq_test": seq_test,
#             "seg_train": seg_train,
#             "seq_train": seq_train,
#             "seg_val": seg_val,
#             "seq_val": seq_val}
# write_pickle(DWI_data, output_file)

EOFError: Ran out of input

# T2

In [60]:
input_dir = Path("../ROI/ROI_NEW/T2")
output_file = Path("../data/pickles/T2.pkl")
seqs, segs = get_seqs(input_dir)
seqs, segs = transpose(seqs, segs)
seqs, segs = convert_to_tensor(seqs, segs)
seqs = normalize_by(seqs, max_v=1370.8, mean=123.746, std=207.8, stds=6, clip=True)
seg_train, seg_test, seg_val = train_test_validation(segs)
seq_train, seq_test, seq_val = train_test_validation(seqs)
T2_data = {"seg_test": seg_test,
            "seq_test": seq_test,
            "seg_train": seg_train,
            "seq_train": seq_train,
            "seg_val": seg_val,
            "seq_val": seq_val}
write_pickle(T2_data, output_file)

# T2star

In [20]:
input_dir = Path("../ROI/ROI_NEW/T2star")
output_file = Path("../data/pickles/T2star.pkl")
seqs, segs = get_seqs(input_dir)
# seqs, segs = transpose(seqs, segs)
seqs, segs = allow_only_by(seqs, segs)

seqs, segs = convert_to_tensor(seqs, segs)
# print(len(seqs), len(segs))
seqs = normalize_by(seqs, max_v=300, mean=133.748, std=0, stds=0, clip_max=300)
seg_train, seg_test, seg_val = train_test_validation(segs)
seq_train, seq_test, seq_val = train_test_validation(seqs)
T2star_data = {"seg_test": seg_test,
            "seq_test": seq_test,
            "seg_train": seg_train,
            "seq_train": seq_train,
            "seg_val": seg_val,
            "seq_val": seq_val}
write_pickle(T2star_data, output_file)